In [1]:
print("this")

this


In [2]:
import math
import string
import nltk
string.punctuation
stop_list=nltk.corpus.stopwords.words('english')
def rem_punctuation(text):
    txt_np="".join([c for c in text if c not in string.punctuation])
    return txt_np
class BooleanModel():
    
    @staticmethod
    def and_operation(left_operand, right_operand):
        # perform 'merge'
        result = []                                 # results list to be returned
        l_index = 0                                 # current index in left_operand
        r_index = 0                                 # current index in right_operand
        jump_l = int(math.sqrt(len(left_operand)))  # skip pointer distance for l_index
        jump_r = int(math.sqrt(len(right_operand))) # skip pointer distance for r_index

        while (l_index < len(left_operand) and r_index < len(right_operand)):
            l_item = left_operand[l_index]  # current item in left_operand
            r_item = right_operand[r_index] # current item in right_operand
            
            # case 1: if match
            if (l_item == r_item):
                result.append(l_item)   # add to results
                l_index += 1            # advance left index
                r_index += 1            # advance right index
            
            # case 2: if left item is more than right item
            elif (l_item > r_item):
                # if r_index can be skipped (if new r_index is still within range and resulting item is <= left item)
                if (r_index + jump_r < len(right_operand)) and right_operand[r_index + jump_r] <= l_item:
                    r_index += jump_r
                # else advance r_index by 1
                else:
                    r_index += 1

            # case 3: if left item is less than right item
            else:
                # if l_index can be skipped (if new l_index is still within range and resulting item is <= right item)
                if (l_index + jump_l < len(left_operand)) and left_operand[l_index + jump_l] <= r_item:
                    l_index += jump_l
                # else advance l_index by 1
                else:
                    l_index += 1

        return result

    @staticmethod
    def or_operation(left_operand, right_operand):
        result = []     # union of left and right operand
        l_index = 0     # current index in left_operand
        r_index = 0     # current index in right_operand

        # while lists have not yet been covered
        while (l_index < len(left_operand) or r_index < len(right_operand)):
            # if both list are not yet exhausted
            if (l_index < len(left_operand) and r_index < len(right_operand)):
                l_item = left_operand[l_index]  # current item in left_operand
                r_item = right_operand[r_index] # current item in right_operand
                
                # case 1: if items are equal, add either one to result and advance both pointers
                if (l_item == r_item):
                    result.append(l_item)
                    l_index += 1
                    r_index += 1

                # case 2: l_item greater than r_item, add r_item and advance r_index
                elif (l_item > r_item):
                    result.append(r_item)
                    r_index += 1

                # case 3: l_item lower than r_item, add l_item and advance l_index
                else:
                    result.append(l_item)
                    l_index += 1

            # if left_operand list is exhausted, append r_item and advance r_index
            elif (l_index >= len(left_operand)):
                r_item = right_operand[r_index]
                result.append(r_item)
                r_index += 1

            # else if right_operand list is exhausted, append l_item and advance l_index 
            else:
                l_item = left_operand[l_index]
                result.append(l_item)
                l_index += 1

        return result

    @staticmethod
    def not_operation(right_operand, indexed_docIDs):
        # complement of an empty list is list of all indexed docIDs
        if (not right_operand):
            return indexed_docIDs
        
        result = []
        r_index = 0 # index for right operand
        for item in indexed_docIDs:
            # if item do not match that in right_operand, it belongs to compliment 
            if (item != right_operand[r_index]):
                result.append(item)
            # else if item matches and r_index still can progress, advance it by 1
            elif (r_index + 1 < len(right_operand)):
                r_index += 1
        return result

In [3]:
from __future__ import print_function


class Node(object):
    """Tree node: left and right child + data which can be any object
    """
    def __init__(self, data):
        """Node constructor
        @param data node data object
        """
        self.left = None
        self.right = None
        self.data = data

    def insert(self, data):
        """Insert new node with data
        @param data node data object to insert
        """
        if self.data:
            if data < self.data:
                if self.left is None:
                    self.left = Node(data)
                else:
                    self.left.insert(data)
            elif data > self.data:
                if self.right is None:
                    self.right = Node(data)
                else:
                    self.right.insert(data)
        else:
            self.data = data

    def lookup(self, data, parent=None):
        """Lookup node containing data
        @param data node data object to look up
        @param parent node's parent
        @returns node and node's parent if found or None, None
        """
        if data < self.data:
            if self.left is None:
                return None, None
            return self.left.lookup(data, self)
        elif data > self.data:
            if self.right is None:
                return None, None
            return self.right.lookup(data, self)
        else:
            return self, parent

    def delete(self, data):
        """Delete node containing data
        @param data node's content to delete
        """
        # get node containing data
        node, parent = self.lookup(data)
        if node is not None:
            children_count = node.children_count()
            if children_count == 0:
                # if node has no children, just remove it
                if parent:
                    if parent.left is node:
                        parent.left = None
                    else:
                        parent.right = None
                else:
                    self.data = None
            elif children_count == 1:
                # if node has 1 child
                # replace node by its child
                if node.left:
                    n = node.left
                else:
                    n = node.right
                if parent:
                    if parent.left is node:
                        parent.left = n
                    else:
                        parent.right = n
                else:
                    self.left = n.left
                    self.right = n.right
                    self.data = n.data
            else:
                # if node has 2 children
                # find its successor
                parent = node
                successor = node.right
                while successor.left:
                    parent = successor
                    successor = successor.left
                # replace node data by its successor data
                node.data = successor.data
                # fix successor's parent node child
                if parent.left == successor:
                    parent.left = successor.right
                else:
                    parent.right = successor.right

    def compare_trees(self, node):
        """Compare 2 trees
        @param node tree to compare
        @returns True if the tree passed is identical to this tree
        """
        if node is None:
            return False
        if self.data != node.data:
            return False
        res = True
        if self.left is None:
            if node.left:
                return False
        else:
            res = self.left.compare_trees(node.left)
        if res is False:
            return False
        if self.right is None:
            if node.right:
                return False
        else:
            res = self.right.compare_trees(node.right)
        return res

    def print_tree(self):
        """Print tree content inorder
        """
        if self.left:
            self.left.print_tree()
        print(self.data, end=" ")
        if self.right:
            self.right.print_tree()

    def tree_data(self):
        """Generator to get the tree nodes data
        """
        # we use a stack to traverse the tree in a non-recursive way
        stack = []
        node = self
        while stack or node:
            if node:
                stack.append(node)
                node = node.left
            else:
                # we are returning so we pop the node and we yield it
                node = stack.pop()
                yield node.data
                node = node.right

    def children_count(self):
        """Return the number of children
        @returns number of children: 0, 1, 2
        """
        cnt = 0
        if self.left:
            cnt += 1
        if self.right:
            cnt += 1
        return cnt

In [4]:
import nltk
import collections

#from boolean import BooleanModel

# Build from sources: https://github.com/laurentluce/python-algorithms


class IRSystem():

    def __init__(self, docs=None, stop_words=stop_list):
        if docs is None:
            raise UserWarning('Docs should not be none')
        self._docs = docs
        self._stemmer = nltk.stem.porter.PorterStemmer()
        self._inverted_index = self._preprocess_corpus(stop_words)
        self._print_inverted_index()

    def _preprocess_corpus(self,stop_words=stop_list):
        index = {}
        for i, doc in enumerate(self._docs):
            for word in doc.split():
                token = self._stemmer.stem(word.lower())
                if token in stop_words:
                    continue
                token=rem_punctuation(token)    
                #token = self._stemmer.stem(word.lower())
                if index.get(token, -244) == -244:
                    index[token] = Node(i + 1)
                elif isinstance(index[token], Node):
                    index[token].insert(i + 1)
                else:
                    raise UserWarning('Wrong data type for posting list')
        return index

    def _print_inverted_index(self):
        print('INVERTED INDEX:\n')
        for word, tree in self._inverted_index.items():
            print('{}: {}'.format(word, [doc_id for doc_id in tree.tree_data() if doc_id != None]))
        print()

    def _get_posting_list(self, word):
        return [doc_id for doc_id in self._inverted_index[word].tree_data() if doc_id != None]

    @staticmethod
    def _parse_query(infix_tokens):
        """ Parse Query 
        Parsing done using Shunting Yard Algorithm 
        """
        precedence = {}
        precedence['NOT'] = 3
        precedence['AND'] = 2
        precedence['OR'] = 1
        precedence['('] = 0
        precedence[')'] = 0    

        output = []
        operator_stack = []

        for token in infix_tokens:
            if (token == '('):
                operator_stack.append(token)
            
            # if right bracket, pop all operators from operator stack onto output until we hit left bracket
            elif (token == ')'):
                operator = operator_stack.pop()
                while operator != '(':
                    output.append(operator)
                    operator = operator_stack.pop()
            
            # if operator, pop operators from operator stack to queue if they are of higher precedence
            elif (token in precedence):
                # if operator stack is not empty
                if (operator_stack):
                    current_operator = operator_stack[-1]
                    while (operator_stack and precedence[current_operator] > precedence[token]):
                        output.append(operator_stack.pop())
                        if (operator_stack):
                            current_operator = operator_stack[-1]
                operator_stack.append(token) # add token to stack
            else:
                output.append(token.lower())

        # while there are still operators on the stack, pop them into the queue
        while (operator_stack):
            output.append(operator_stack.pop())

        return output

    def process_query(self, query):
        # prepare query list
        query = query.replace('(', '( ')
        query = query.replace(')', ' )')
        query = query.split(' ')

        indexed_docIDs = list(range(1, len(self._docs) + 1))

        results_stack = []
        postfix_queue = collections.deque(self._parse_query(query)) # get query in postfix notation as a queue

        while postfix_queue:
            token = postfix_queue.popleft()
            result = [] # the evaluated result at each stage
            # if operand, add postings list for term to results stack
            if (token != 'AND' and token != 'OR' and token != 'NOT'):
                token = self._stemmer.stem(token) # stem the token
                # default empty list if not in dictionary
                if (token in self._inverted_index):
                    result = self._get_posting_list(token)
            
            elif (token == 'AND'):
                right_operand = results_stack.pop()
                left_operand = results_stack.pop()
                result = BooleanModel.and_operation(left_operand, right_operand)   # evaluate AND

            elif (token == 'OR'):
                right_operand = results_stack.pop()
                left_operand = results_stack.pop()
                result = BooleanModel.or_operation(left_operand, right_operand)    # evaluate OR

            elif (token == 'NOT'):
                right_operand = results_stack.pop()
                result = BooleanModel.not_operation(right_operand, indexed_docIDs) # evaluate NOT

            results_stack.append(result)                        

        # NOTE: at this point results_stack should only have one item and it is the final result
        if len(results_stack) != 1: 
            print("ERROR: Invalid Query. Please check query syntax.") # check for errors
            return None
        
        return results_stack.pop()

In [ ]:
import argparse
import timeit
from easydict import EasyDict

import os
import pandas as pd
from nltk.tokenize import word_tokenize


inverted = dict()
docs=list()
for filename in os.listdir("E:\Info_Retreival\data"):
    with open(os.path.join("E:\Info_Retreival\data", filename), 'r') as f:
        
        text = f.read()
        docs.append(text)
        
#docs = ['hello i m a machine learning engineer haha', 
        #'hello bad world machine engineering people', 
        #'the world is a bad place',
        #'engineering a great machine that learns']


#stop_words = ['is', 'a', 'for', 'the', 'of']



def main():
    
    
    ir = IRSystem(docs, stop_words=stop_list)

    while True:
        query = input('Enter boolean query: ')

        
        results = ir.process_query(query)
        
        if results is not None:
            
            print('\nDoc IDS: ')
            print(results)
        print()
        
        
if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt as e:
        print('EXIT')
   

INVERTED INDEX:

midsumm: [1, 4, 41]
night: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
dream: [1, 2, 3, 4, 6, 7, 9, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41]
william: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
shakespear: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
edit: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
barbara: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,

muddy: [1, 33, 34, 36, 37]
lips: [1, 3, 4, 5, 6, 7, 8, 11, 12, 14, 17, 18, 19, 20, 21, 23, 24, 26, 27, 28, 30, 31, 36, 37, 38, 39, 40, 41, 42]
kiss: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
cherries: [1, 42]
pure: [1, 2, 3, 4, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 34, 35, 36, 37, 38, 39, 40, 41, 42]
congeal: [1, 3, 9, 12, 16, 19, 21, 25, 33, 42]
white: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
taurus: [1, 3, 39, 41]
snow: [1, 2, 3, 5, 6, 7, 11, 14, 16, 17, 18, 19, 20, 23, 25, 26, 27, 28, 29, 30, 31, 34, 35, 36, 37, 38, 39, 42]
eastern: [1, 3, 19, 26, 27, 40]
holdst: [1, 7, 9, 24]
princess: [1, 3, 4, 6, 10, 11, 13, 14, 16, 17, 18, 19, 24, 25, 34, 37, 41]
bliss: [1, 3, 10, 11, 12, 17, 19, 21, 23, 27, 28, 29, 30, 33, 36, 39]


thrice: [2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 17, 18, 20, 22, 26, 27, 28, 30, 31, 33, 35, 36, 37, 38, 40, 41, 42]
displeasure: [2, 6, 9, 13, 17, 22, 23, 24, 27, 30, 37, 38]
boot: [2, 3, 6, 8, 9, 10, 11, 12, 14, 17, 20, 21, 24, 25, 26, 28, 31, 33, 35, 36, 37, 39, 40, 41]
spur: [2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 17, 18, 20, 21, 24, 25, 26, 28, 34, 35, 36, 37, 38, 40, 41, 42]
leapt: [2, 9, 17, 19, 22, 27, 28, 39]
custard: [2]
residence: [2, 7, 16, 26, 38]
nut: [2, 4, 29, 40]
clothes: [2, 6, 8, 17, 21, 23, 24, 27, 31, 33, 37, 38]
consequence: [2, 7, 14, 15, 20, 21, 26, 30]
naturesfarewell: [2]
clog: [2, 20, 22, 23, 26, 36, 37, 41]
procur: [2, 6, 8, 9, 10, 11, 13, 17, 21, 23, 26, 27, 29, 31, 33, 36, 37, 40]
parting: [2, 33, 35, 36]
course: [2, 3, 6, 7, 8, 10, 12, 14, 15, 16, 17, 18, 20, 21, 22, 23, 25, 26, 27, 29, 30, 33, 34, 35, 36, 38, 39, 40]
ministr: [2, 7]
unsettled: [2, 37]
drive: [2, 3, 5, 6, 7, 8, 9, 10, 13, 15, 16, 17, 19, 22, 24, 26, 27, 29, 30, 31, 34, 35, 38, 39, 41, 

gests: [3]
spill: [3, 7, 10, 16, 17, 18, 19, 26, 27, 38, 42]
escaped: [3, 16, 36]
doughtyhand: [3]
hectors: [3, 18, 40]
clip: [3, 5, 6, 8, 11, 18, 23, 24, 36, 37, 42]
feats: [3, 5, 7, 10, 14, 24]
gash: [3, 5, 8, 14, 20, 24, 40, 42]
theeo: [3]
chain: [3, 4, 5, 9, 10, 11, 12, 18, 22, 23, 26, 27, 29, 31, 34, 35, 39, 40, 41, 42]
attir: [3, 6, 11, 19, 22, 27, 31, 34, 39, 41]
proof: [3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 17, 20, 21, 22, 23, 25, 26, 27, 28, 30, 31, 33, 35, 36, 37, 38, 40, 41, 42]
har: [3, 8, 16, 20, 33, 38, 40]
pant: [3, 8, 9, 17, 19, 23, 25, 38, 41, 42]
triumphing: [3, 25]
uncaught: [3, 17]
beds: [3, 6, 7, 10, 20, 23, 24, 26, 28, 31, 36, 41]
nourish: [3, 5, 8, 10, 11, 12, 13, 18, 20, 23, 27, 28, 35, 36, 37, 39]
nerv: [3, 5, 20, 21, 28, 34, 36, 40]
goal: [3, 24, 37]
mankind: [3, 5, 20, 31, 34, 36, 37, 38, 40, 41]
carbuncl: [3, 5, 6, 17]
jolli: [3, 6, 16, 17, 25, 33, 36, 41]
hack: [3, 8, 10, 15, 25, 26, 31, 40]
capac: [3, 5, 9, 13, 18, 31, 40, 41]
sup: [3, 5, 8, 9, 11, 15, 18, 

pleaders: [5]
disproperti: [5]
freedoms: [5]
camel: [5, 7, 26, 40]
provand: [5]
burdens: [5, 11, 30]
soar: [5, 6, 11, 14, 15, 27, 33]
peoplewhich: [5]
sheepwil: [5]
stubble: [5]
flung: [5, 13, 20, 34, 38]
gloves: [5, 14, 23, 30, 31, 37]
handkerchiefs: [5]
statue: [5, 15, 37]
shouts: [5, 15]
consulships: [5]
carelessness: [5]
indiffer: [5, 7, 13, 14, 26, 33, 39, 40, 41]
opposite: [5, 9, 17, 21, 25, 41]
dislikes: [5, 12]
ascent: [5]
suppl: [5, 26, 27, 38, 40]
bonneted: [5]
ingrat: [5, 8, 16, 17, 38, 39, 41]
remains: [5, 6, 7, 8, 10, 11, 12, 21, 22, 26, 28, 36]
aftermeeting: [5]
gratifi: [5, 6, 18, 23, 24, 30, 33, 39]
wellfound: [5, 36]
sits: [5, 7, 9, 12, 13, 17, 19, 22, 24, 26, 42]
requital: [5, 21]
kindest: [5, 30, 33]
convent: [5, 13, 21, 36]
hereto: [5]
willingly: [5, 6, 8, 11, 22, 24, 41]
pertin: [5]
disbench: [5]
idli: [5, 9, 10, 11, 14, 16, 26, 28, 29, 33, 38, 40]
monstered: [5]
spawn: [5, 21]
onewhen: [5]
itproceed: [5]
feebly: [5]
chiefest: [5, 7, 10, 11, 12, 16, 20, 24, 25, 30,

...

cuts: [22]
underborn: [22]
bluish: [22]
tinsel: [22]
graceful: [22]
barns: [22]
carduu: [22]
benedictu: [22]
prickst: [22]
benedictus: [22]
grudging: [22]
headborough: [22, 33]
decern: [22]
odorous: [22]
palabras: [22]
seewel: [22]
vergeswell: [22]
worshiped: [22]
aspici: [22]
suffigance: [22]
noncome: [22]
herfriar: [22]
interjections: [22]
friarfather: [22]
unconstrain: [22]
orang: [22]
sensuality: [22, 23]
heros: [22]
fatherli: [22, 33]
maidenleonato: [22]
themthus: [22]
misgovernment: [22]
thinkhelp: [22]
reproaches: [22]
frugal: [22, 31]
herwhi: [22]
observations: [22]
nakedness: [22, 38]
mercyo: [22]
idea: [22, 25]
reclus: [22]
consented: [22, 31]
slandered: [22]
unmitig: [22, 34]
rancoro: [22]
comfect: [22]
grieving: [22, 23, 30]
examine: [22]
borachioyours: [22]
borachioconrad: [22]
villainsmasters: [22]
himcom: [22]
asidefor: [22]
eftest: [22]
waylet: [22]
burglari: [22]
diedmast: [22]
opinioned: [22]
themthou: [22]
assno: [22]
himbr: [22]
awayo: [22]
profitless: [22, 23, 28]


employments: [26]
elsewher: [26, 28, 33]
soonbeliev: [26]
consequently: [26]
sluic: [26, 37]
abels: [26]
soars: [26]
unstoop: [26]
appealed: [26]
wrathkindl: [26]
prescribe: [26]
makepeac: [26]
impeached: [26]
withstood: [26]
tentimesbarredup: [26]
crestfallen: [26, 31]
beggarfear: [26]
recant: [26, 30, 38]
lambert: [26]
officersatarm: [26]
lieth: [26, 39]
faded: [26, 33, 34]
breathest: [26]
pathway: [26, 27]
himah: [26]
plashi: [26]
untrodden: [26]
aumerl: [26]
violate: [26]
formally: [26]
daringhardi: [26]
bloodwhich: [26]
regener: [26]
spears: [26]
fostered: [26, 35]
eaglewing: [26]
skyaspir: [26]
rivalh: [26]
dateless: [26, 27, 28]
unstring: [26]
enjail: [26]
portcullis: [26]
unfeel: [26, 29, 30]
administer: [26]
oildri: [26]
timebewast: [26]
farewelland: [26]
returnest: [26]
winters: [26, 34]
miscal: [26, 28, 40]
apprenticehood: [26]
journeyman: [26]
havens: [26]
treadst: [26]
strewed: [26, 33]
caucasus: [26, 39]
wallow: [26, 40]
lanceth: [26]
observecousin: [26]
northeast: [26, 3

digestions: [29]
brawls: [29, 39]
lifepreserv: [29]
syrups: [29]
arow: [29]
scissor: [29, 36]
nothingguard: [29]
faultsay: [29]
womanthey: [29]
chargeth: [29]
headyrash: [29]
hungry: [29, 36, 37, 38]
anatomy: [29, 41]
threadbar: [29]
fortuneteller: [29]
needy: [29]
holloweyed: [29]
sharplook: [29]
dankish: [29]
intric: [29]
circe: [29]
coldly: [29, 33, 37]
strangego: [29]
defeatur: [29]
denies: [29]
sapconsum: [29]
witnessesi: [29]
masterwho: [29]
dromios: [29]
raft: [29]
menaphon: [29]
arose: [29]
thirtythre: [29]
shipboard: [29, 37]
httpsshakespearefolgeredushakespearesworksthemerchantofvenice: [30]
heiress: [30]
belmont: [30]
nerissa: [30]
balthazar: [30]
stephano: [30, 34]
morocco: [30]
arragon: [30]
bassanio: [30]
solanio: [30]
salarino: [30]
lorenzo: [30]
leonardo: [30]
shylock: [30]
jewish: [30]
moneylend: [30]
jessica: [30]
tubal: [30]
lancelet: [30]
gobbo: [30]
salerio: [30]
wantwit: [30]
traffick: [30]
pire: [30]
andrew: [30, 41]
dock: [30]
enrob: [30]
silks: [30]
twohead: [3

perished: [34, 36]
amazement: [34, 37, 40]
artwip: [34]
heardst: [34, 35]
inquisition: [34, 40]
issued: [34]
perfidioush: [34]
signori: [34]
heedfully: [34]
overtopping: [34]
verdur: [34, 42]
attendst: [34]
oerpr: [34]
lorded: [34]
incapable: [34]
unbowedalas: [34]
inveterate: [34]
impertinent: [34]
groaned: [34]
linens: [34]
stuffs: [34]
necessaries: [34]
seasorrow: [34]
vainer: [34]
mindyour: [34]
zenith: [34]
yards: [34]
bowsprit: [34]
precursor: [34]
thunderclaps: [34]
sightoutrun: [34]
upstaringthen: [34]
reeds: [34]
leaped: [34]
calledst: [34]
stillvex: [34]
bermoothes: [34]
preciously: [34]
mistakings: [34]
grumblings: [34]
sycorax: [34]
argier: [34]
sorceri: [34]
blueey: [34]
hagborn: [34]
everangri: [34]
murmurst: [34]
uswhat: [34]
tortoise: [34]
cramps: [34]
sidestitch: [34]
urchin: [34]
honeycomb: [34]
strokst: [34]
pits: [34, 42]
bats: [34]
calibans: [34]
deservedli: [34]
hagseed: [34]
shruggst: [34]
neglectst: [34]
setebos: [34]
whist: [34]
featli: [34]
dispersedly: [34]
b

library: [39]
tosseth: [39]
metamorphosis: [39]
findlavinia: [39]
describes: [39]
rapes: [39]
tragedies: [39]
slunk: [39]
niecebrother: [39]
herelook: [39]
stuprum: [39]
magni: [39]
poli: [39]
lentu: [39]
audi: [39]
scelera: [39]
vides: [39]
melavinia: [39]
meas: [39]
bearwhelps: [39]
playeth: [39]
thenboy: [39]
bedchamb: [39, 42]
comemarcus: [39]
deciphered: [39]
rapemay: [39]
armori: [39]
bothaside: [39]
integ: [39]
scelerisqu: [39]
purus: [39]
eget: [39]
mauri: [39]
iaculis: [39]
arcu: [39]
horace: [39]
caterwaul: [39, 41]
blowse: [39]
tadpol: [39]
younglings: [39]
enceladu: [39]
typhon: [39, 40]
sanguine: [39]
shallowheart: [39]
whitelim: [39]
ignomy: [39]
wheak: [39]
muliteu: [39]
whey: [39]
reliquit: [39]
tools: [39]
suffrag: [39]
manofwar: [39]
unsearched: [39]
shrubs: [39]
bigbon: [39]
jovem: [39]
youhere: [39]
apollinem: [39]
martem: [39]
pallashere: [39]
caiusnot: [39]
boymarcus: [39]
unsolicited: [39]
virgo: [39]
ari: [39]
gibbetmaker: [39]
jubiter: [39]
plebs: [39]
emperal: